IAMSI -- 2017-2018
--------
*&copy;Equipe pédagogique IAMSI'2018*


# TME 09 : Règles d'association

<font size="+1">**[Q]**</font> **Indiquer dans la boîte ci-dessous vos noms et prénoms :**

Kherfallah Celia, Fernandez Stieban

## Présentation

### But de ce TME

Ce TME a pour but de réaliser une implémentation *intuitive* de l'algorithme **apriori** afin de la comparer à une version efficace de cet algorithme, puis à une implémentation de l'algorithme **fp-growth**.


#### Compte-rendu de la séance

Le compte-rendu de ce TME se compose de ce fichier ipython complété par les réponses aux questions posées.

Ce compte-rendu est à poster : 
- à l'issue de la séance, un premier envoi doit être **obligatoirement** fait avec ce que vous avez réalisé
- si nécessaire, un second envoi peut être fait **au plus tard avant le début de la prochaine séance**.


<font color="RED">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : *Nom1_Nom2.ipynb* 
- *Nom1* et *Nom2* : noms des membres du binôme
- ne pas compresser ou faire une archive: envoyez le fichier ipython tel quel, éventuellement, si vous avez d'autres fichiers à envoyer, les  joindre au message.

#### A télécharger

Avant de commencer, télécharger l'archive **tme-09.tgz** à l'adresse indiquée.

Une fois détarée, cette archive crée un répertoire contenant :
- deux fichiers exécutables (Linux): *apriori32* et *fpgrowth32* (ces 2 programmes ont été développés par Christian Borgelt, chercheur au ``European Center of Soft Computing" , plus d'infos sur sa page web  http://www.borgelt.net/software.html). Ces programmes seront utilisés en fin de séance.
- un répertoire *datasets* contenant des bases d'exemples pour tester les algorithmes
- ce fichier ipython notebook que vous lisez en ce moment et que vous allez compléter.



## Traitement d'une base d'apprentissage

### Chargement de la base

On commence par travailler sur la base exemple du fichier "*exemple-1.txt*" (fourni dans le répertoire datasets). 

Ce fichier contient une transaction par ligne. Chaque transaction est composée d'un groupe d'items séparés par un espace.

On peut charger en Python ce fichier par la commande suivante (le répertoire datasets doit se trouver dans le répertoire courant) :

In [12]:
import csv
with open('datasets/exemple-1.txt', 'r') as fichier:
    lecteur = csv.reader(fichier, delimiter=' ')
    i = 0
    for ligne in lecteur:
        i += 1
        print('ligne',i,':',ligne)

ligne 1 : ['a', 'b', 'c']
ligne 2 : ['a', 'd', 'e']
ligne 3 : ['b', 'c', 'd']
ligne 4 : ['a', 'b', 'c', 'd']
ligne 5 : ['b', 'c']
ligne 6 : ['a', 'b', 'd']
ligne 7 : ['d', 'e']
ligne 8 : ['a', 'b', 'c', 'd']
ligne 9 : ['c', 'd', 'e']
ligne 10 : ['a', 'b', 'c']


<font size="+1">**[Q]**</font> Ecrire la fonction <code>chargeBase</code> qui prend en argument un nom de fichier, respectant le format énoncé plus haut, le lit et rend un dictionnaire dont les clés sont les numéros de ligne (une transaction) et les valeurs associées les itemsets correspondants représentés sous forme d'ensembles Python (des <code>set()</code> donc).

Ici, il est plus intéressant de représenter un itemset comme un ensemble d'items plutôt que comme une liste. Il sera ainsi plus facile de réaliser des comparaisons d'ensembles ou des ajouts d'éléments.

Dans le reste de ce document, on appelle **BASE** un dictionnaire de ce type.

In [13]:
def chargeBase(filename, sep=' '):
    """ str -> BASE
        Retourne un dictionnaire représentant une BASE à partir d'un fichier
    """
    with open(filename, 'r') as fichier:
        lecteur = csv.reader(fichier, delimiter=sep)
        BASE = {}
        for i, ligne in enumerate(lecteur):
            BASE[i+1] = set(ligne)
    return BASE

In [14]:
print("Résultat du chargement de 'exemple-1.txt', on obtient : ")
Base1 = chargeBase('datasets/exemple-1.txt')
Base1

Résultat du chargement de 'exemple-1.txt', on obtient : 


{1: {'a', 'b', 'c'},
 2: {'a', 'd', 'e'},
 3: {'b', 'c', 'd'},
 4: {'a', 'b', 'c', 'd'},
 5: {'b', 'c'},
 6: {'a', 'b', 'd'},
 7: {'d', 'e'},
 8: {'a', 'b', 'c', 'd'},
 9: {'c', 'd', 'e'},
 10: {'a', 'b', 'c'}}

On utilise la variable **Base1** dans la suite pour faire référence à cette base.

## Itemsets et support

<font size="+1">**[Q]**</font> Ecrire la fonction <code>noms_items</code> qui prend en argument une BASE et rend l'ensemble des items qui composent cette base.

In [15]:
def noms_items(BASE):
    """ BASE -> set(str) 
        Retourne l'ensemble des items qui correspondent à BASE 
    """
    return set.union(*BASE.values())

In [16]:
print("Pour la BASE précédente :")
noms_items(Base1)

Pour la BASE précédente :


{'a', 'b', 'c', 'd', 'e'}

<font size="+1">**[Q]**</font> Ecrire la fonction <code>singletons</code> qui prend en argument une BASE et rend la liste des itemsets de taille 1 obtenus à partir de cette base.

Remarque: attention, ici on utilise une **liste** pour stocker les itemsets (un itemset est un ensemble Python) car il n'est pas possible en Python de créer des ensembles d'ensembles.


In [21]:
def singletons(BASE):
    """ BASE -> list(set)
        Retourne une liste d'itemsets de taille 1
    """
    return list(map(lambda x: {x}, noms_items(BASE)))

In [42]:
print("Exemple: singletons(Base1) rend :")
singletons(Base1)

Exemple: singletons(Base1) rend :


[{'b'}, {'d'}, {'e'}, {'a'}, {'c'}]

<font size="+1">**[Q]**</font> Ecrire la fonction <code>comptage</code> qui, pour une BASE et un itemset donnés, rend le nombre de transactions de BASE qui contiennent cet itemset.

In [45]:
def comptage(BASE, itemset):
    """ BASE * itemset -> int 
        Rend le nombre de transactions de BASE qui contiennent itemset
    """
    return len(list(filter(lambda x: itemset.issubset(x), BASE.values())))

In [46]:
print("Comptage de l'itemset {'a','b','c'} dans la base précédente : ")
print("comptage(Base1,{'a','b','c'}) rend la valeur "+str(comptage(Base1,{'a','b','c'})))

Comptage de l'itemset {'a','b','c'} dans la base précédente : 
comptage(Base1,{'a','b','c'}) rend la valeur 4


<font size="+1">**[Q]**</font> Ecrire la fonction <code>support</code> qui, pour une BASE et un itemset donnés, rend le support de cet itemset dans la BASE.

In [47]:
def support(BASE, itemset):
    """ BASE * itemset -> float 
        Retourne le support de itemset dans BASE
    """
    return comptage(BASE, itemset)*1./len(BASE)

In [48]:
print("Support de l'itemset {'a','b','c'} dans la base précédente : "+str(support(Base1,{'a','b','c'})))

Support de l'itemset {'a','b','c'} dans la base précédente : 0.4


## Implémentation de l'algorithme a-priori

Dans cette partie, une implémentation de la partie de construction des itemsets fréquents de l'algorithme a-priori est réalisée. On ne s'intéresse pas dans cette question à la génération des règles d'association (mais cela peut être fait en complément).

Votre programme doit pouvoir s'appliquer aux bases fournies dans le répertoire *datasets* (éventuellement, sur au moins les 10 premiers exemples de la base mushrooms).

<font size="+1">**[Q]**</font> Ecrire la fonction <code>apriori_gen</code> qui prend en argument une liste d'itemsets de même longueur $k$, applique l'algorithme apriori-gen pour rendre la liste des itemsets candidats de longueurs $k+1$.


In [51]:
import itertools as it

def filtreCandidat(candidat, itemsets):
    """ set * list(itemset) -> bool
        Retourne True si tous les subsets de candidat de taille k se retrouvent dans itemsets
        sinon False
    """
    tmp = it.combinations(candidat, len(itemsets[0]))
    for dc in tmp:
        if set(dc) not in itemsets:
            return False
    return True

def apriori_gen(itemsets):
    """ list(itemset) -> list(itemset)
        Retourne la liste des itemsets candidats de longueur k+1 a partir de itemsets
    """
    if not itemsets:
        return []
    k = len(itemsets[0])
    C = it.combinations(itemsets, k+1)
    C = map(lambda x: set.union(*x), C)
    C = list(filter(lambda x: (len(x) == k+1) and filtreCandidat(x, itemsets), C))
    return C
    

In [52]:
########## TEST ##########
print("Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend ")
print(apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]))

print("\nExemple: apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]) rend ")
print(apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]))

Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend 
[{'b', 'a'}, {'c', 'a'}, {'d', 'a'}, {'b', 'c'}, {'b', 'd'}, {'d', 'c'}]

Exemple: apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]) rend 
[{'b', 'd', 'a'}, {'b', 'd', 'c'}]


In [53]:
print("Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend ")
print(apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]))

print("\nExemple: apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]) rend ")
print(apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]))

Exemple: apriori_gen([{'a'}, {'b'}, {'c'}, {'d'}]) rend 
[{'b', 'a'}, {'c', 'a'}, {'d', 'a'}, {'b', 'c'}, {'b', 'd'}, {'d', 'c'}]

Exemple: apriori_gen([{'a', 'b'}, {'a', 'd'}, {'b', 'd'}, {'b', 'c'}, {'c', 'd'}]) rend 
[{'b', 'd', 'a'}, {'b', 'd', 'c'}]


<font size="+1">**[Q]**</font> Ecrire la fonction <code>apriori</code> qui prend en argument une BASE et une valeur réelle comprise entre 0 et 1, et qui rend une liste de tuples dont le premier élément est un itemset trouvé et le deuxième élément est la valeur de support correspondante.

In [54]:
def apriori(BASE, minsup):
    """ BASE * float -> list(tuple)
        Retourne une liste de tuples dont le premier element est un itemset trouve
        et le deuxieme element est la valeur de support correspondante
    """
    res = []
    C = singletons(BASE)
    while C:
        F = []
        for c in C:
            s = support(BASE, c)
            if s >= minsup:
                F.append(c)
                res.append((c, s))
        C = apriori_gen(F)
    return res

In [55]:
########## TEST ##########
print("Exemple: apriori(Base1, 0.3]) rend ")
print(apriori(Base1,0.7))

Exemple: apriori(Base1, 0.3]) rend 
[({'b'}, 0.7), ({'d'}, 0.7), ({'c'}, 0.7)]


In [58]:
print("Exemple: apriori(Base1, 0.3]) rend ")
print(apriori(Base1,0.3))

Exemple: apriori(Base1, 0.3]) rend 
[({'b'}, 0.7), ({'d'}, 0.7), ({'e'}, 0.3), ({'a'}, 0.6), ({'c'}, 0.7), ({'b', 'd'}, 0.4), ({'b', 'a'}, 0.5), ({'b', 'c'}, 0.6), ({'d', 'e'}, 0.3), ({'d', 'a'}, 0.4), ({'d', 'c'}, 0.4), ({'c', 'a'}, 0.4), ({'b', 'd', 'a'}, 0.3), ({'b', 'd', 'c'}, 0.3), ({'b', 'a', 'c'}, 0.4)]


## Expérimentations

### Vérification avec la base du TD

Utilisez votre fonction <code>apriori</code> avec la base de transactions du TD afin de vérifier que vous obtenez les bons itemsets fréquents.

In [69]:
from pprint import pprint
########## TEST ##########
BaseTD = chargeBase("datasets/TD.txt")
print("Exemple: apriori(BaseTD, 0.7) rend ")
pprint(apriori(BaseTD,0.7))

Exemple: apriori(BaseTD, 0.7) rend 
[({'b'}, 0.7), ({'d'}, 0.9)]


In [71]:
print("Exemple: apriori(BaseTD, 0.3) rend ")
pprint(apriori(BaseTD,0.3))

Exemple: apriori(BaseTD, 0.3) rend 
[({'b'}, 0.7),
 ({'d'}, 0.9),
 ({'c'}, 0.5),
 ({'a'}, 0.5),
 ({'e'}, 0.6),
 ({'b', 'd'}, 0.6),
 ({'b', 'c'}, 0.3),
 ({'b', 'a'}, 0.3),
 ({'b', 'e'}, 0.4),
 ({'d', 'c'}, 0.4),
 ({'d', 'a'}, 0.4),
 ({'d', 'e'}, 0.6),
 ({'e', 'a'}, 0.4),
 ({'b', 'd', 'e'}, 0.4),
 ({'d', 'a', 'e'}, 0.4)]


### Comparaisons avec les implémentations apriori32 et fpgrowth

<font size="+1">**[Q]**</font> Tester votre programme sur les 3 bases données dans datasets et confronter vos résultats avec le programme *apriori32*.

In [70]:
import os
BaseM = chargeBase("datasets/mushrooms.txt", sep=',')
BaseT = chargeBase("datasets/titanic-red.csv", sep=',')

- Base exemple-1

In [72]:
print("Exemple: apriori(Base exemple-1, 0.5) rend ")
pprint(apriori(Base1,0.5))

Exemple: apriori(Base exemple-1, 0.5) rend 
[({'b'}, 0.7),
 ({'d'}, 0.7),
 ({'a'}, 0.6),
 ({'c'}, 0.7),
 ({'b', 'a'}, 0.5),
 ({'b', 'c'}, 0.6)]


In [82]:
os.system("./apriori32 -s50 datasets/exemple-1.txt res.txt")
print(open("res.txt", "r").read())

a (60)
a b (50)
b (70)
b c (60)
c (70)
d (70)



Les résultats sont identiques, notre programme semble fonctionner. Il ne semble pas vraiment y avoir de différence entre les temps d'exécution pour le moment.

- Base TD

In [73]:
print("Exemple: apriori(Base TD, 0.6) rend ")
pprint(apriori(BaseTD,0.6))

Exemple: apriori(Base TD, 0.6) rend 
[({'b'}, 0.7), ({'d'}, 0.9), ({'e'}, 0.6), ({'b', 'd'}, 0.6), ({'d', 'e'}, 0.6)]


In [89]:
os.system("./apriori32 -s60 datasets/TD.txt res.txt")
print(open("res.txt", "r").read())

e d (60)
e (60)
b (70)
b d (60)
d (90)



Encore une fois, résultats similaires mais pas de différence au niveau du temps d'exécution.

- Base Mushrooms

In [75]:
print("Exemple: apriori(Base Mushrooms, 0.5) rend ")
pprint(apriori(BaseM,0.5))

Exemple: apriori(Base Mushrooms, 0.5) rend 
[({'TAPERING'}, 0.5779467680608364),
 ({'FREE'}, 0.9743346007604563),
 ({'PARTIAL'}, 1.0),
 ({'BROWN'}, 0.5881653992395437),
 ({'SMOOTH'}, 0.7873098859315589),
 ({'NO'}, 0.5988593155893536),
 ({'CLOSE'}, 0.8108365019011406),
 ({'EDIBLE'}, 0.5332699619771863),
 ({'ONE'}, 0.9230038022813688),
 ({'WHITE'}, 0.9771863117870723),
 ({'BROAD'}, 0.6986692015209125),
 ({'FREE', 'TAPERING'}, 0.5779467680608364),
 ({'PARTIAL', 'TAPERING'}, 0.5779467680608364),
 ({'SMOOTH', 'TAPERING'}, 0.5370722433460076),
 ({'ONE', 'TAPERING'}, 0.5779467680608364),
 ({'WHITE', 'TAPERING'}, 0.5779467680608364),
 ({'PARTIAL', 'FREE'}, 0.9743346007604563),
 ({'FREE', 'BROWN'}, 0.563925855513308),
 ({'SMOOTH', 'FREE'}, 0.7644961977186312),
 ({'NO', 'FREE'}, 0.5731939163498099),
 ({'FREE', 'CLOSE'}, 0.785171102661597),
 ({'EDIBLE', 'FREE'}, 0.5104562737642585),
 ({'FREE', 'ONE'}, 0.9001901140684411),
 ({'FREE', 'WHITE'}, 0.9743346007604563),
 ({'FREE', 'BROAD'}, 0.6730038022

In [91]:
os.system("./apriori32 -s50 datasets/mushrooms.txt res.txt")
print(open("res.txt", "r").read())

EDIBLE PARTIAL (53.327)
EDIBLE (53.327)
EDIBLE SMOOTH PARTIAL (51.0694)
EDIBLE SMOOTH (51.0694)
EDIBLE FREE PARTIAL WHITE (51.0456)
EDIBLE FREE PARTIAL (51.0456)
EDIBLE FREE WHITE (51.0456)
EDIBLE FREE (51.0456)
EDIBLE WHITE PARTIAL (51.0456)
EDIBLE WHITE (51.0456)
TAPERING PARTIAL ONE FREE WHITE (57.7947)
TAPERING PARTIAL ONE FREE (57.7947)
TAPERING PARTIAL ONE WHITE (57.7947)
TAPERING PARTIAL ONE (57.7947)
TAPERING PARTIAL FREE WHITE (57.7947)
TAPERING PARTIAL FREE (57.7947)
TAPERING PARTIAL WHITE (57.7947)
TAPERING PARTIAL (57.7947)
TAPERING ONE FREE WHITE (57.7947)
TAPERING ONE FREE (57.7947)
TAPERING ONE WHITE (57.7947)
TAPERING ONE (57.7947)
TAPERING FREE WHITE (57.7947)
TAPERING FREE (57.7947)
TAPERING WHITE (57.7947)
TAPERING (57.7947)
TAPERING SMOOTH PARTIAL ONE FREE WHITE (53.7072)
TAPERING SMOOTH PARTIAL ONE FREE (53.7072)
TAPERING SMOOTH PARTIAL ONE WHITE (53.7072)
TAPERING SMOOTH PARTIAL ONE (53.7072)
TAPERING SMOOTH PARTIAL FREE WHITE (53.7072)
TAPERING SMOOTH PARTIAL FRE

- Base titanic

In [76]:
print("Exemple: apriori(Base Titanic, 0.5) rend ")
pprint(apriori(BaseT,0.5))

Exemple: apriori(Base Titanic, 0.5) rend 
[({'Southampton'}, 0.7112010796221323),
 ({'dead'}, 0.5344129554655871),
 ({'male'}, 0.6099865047233468)]


In [93]:
os.system("./apriori32 -s50 datasets/titanic-red.csv res.txt")
print(open("res.txt", "r").read())

dead (53.4413)
male (60.9987)
Southampton (71.1201)



<font size="+1">**[Q]**</font> Utiliser le programme *apriori32* pour générer des règles d'association (voir les différentes options en annexe) sur les différentes bases de données fournies. Tester différents seuils (support et confiance). 

In [94]:
os.system("./apriori32 -trs50m2n5 datasets/titanic-red.csv res.txt")
print(open("res.txt", "r").read())

male <- dead (53.4413, 87.8788)



In [95]:
os.system("./apriori32 -trs30m2n5 datasets/titanic-red.csv res.txt")
print(open("res.txt", "r").read())

Southampton <- 2nd (34.8178, 87.9845)
alive <- female (38.8664, 83.3333)
male <- dead Southampton (41.2955, 89.2157)
male <- dead (53.4413, 87.8788)



In [96]:
os.system("./apriori32 -trs30m2n5c20 datasets/titanic-red.csv res.txt")
print(open("res.txt", "r").read())

Cherbourg <- 1st (37.9217, 43.7722)
Cherbourg <- female alive (32.3887, 32.0833)
Cherbourg <- female (38.8664, 29.1667)
Cherbourg <- alive (46.4238, 31.686)
Cherbourg <- male (60.9987, 20.354)
3rd <- female (38.8664, 22.5694)
3rd <- dead male Southampton (36.8421, 33.3333)
3rd <- dead male (46.9636, 33.046)
3rd <- dead Southampton (41.2955, 35.2941)
3rd <- dead (53.4413, 36.6162)
3rd <- male Southampton (46.2888, 31.7784)
3rd <- male (60.9987, 30.0885)
3rd <- Southampton (71.1201, 27.5142)
2nd <- female alive (32.3887, 38.3333)
female <- 2nd Southampton (30.6343, 40.5286)
female <- 2nd (34.8178, 40.6977)
2nd <- female (38.8664, 36.4583)
alive <- 2nd Southampton (30.6343, 43.6123)
alive <- 2nd (34.8178, 45.3488)
2nd <- alive (46.4238, 34.0116)
2nd <- dead male Southampton (36.8421, 42.1245)
2nd <- dead male (46.9636, 36.7816)
dead <- 2nd Southampton (30.6343, 56.3877)
2nd <- dead Southampton (41.2955, 41.8301)
dead <- 2nd (34.8178, 54.6512)
2nd <- dead (53.4413, 35.6061)
male <- 2nd Sou

In [147]:
os.system("apriori.exe -trs50m2n5c80 datasets/titanic-red.csv res.txt")
print(open("res.txt", "r").read())

male <- dead (53.4413, 87.8788)



<font size="+1">**[Q]**</font>  Ajouter les deux mesures d'intérêt vues en TD (lift et RR) et afficher leurs valeurs pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.


- Base exemple-1

In [245]:
# changement du nom de l'exécutable -> passage à windows
os.system('apriori.exe -trs10m2n5 -el -v " (sup: %S, conf: %C, lift-value: %L, RR: %E)" datasets/exemple-1.txt res.txt')
print(open("res.txt", "r").read())

d <- e a (sup: 10, conf: 100, lift-value: 142.857, RR: 100)
d <- e c (sup: 10, conf: 100, lift-value: 142.857, RR: 100)
d <- e (sup: 30, conf: 100, lift-value: 142.857, RR: 100)
b <- a c d (sup: 20, conf: 100, lift-value: 142.857, RR: 100)
c <- a b (sup: 40, conf: 80, lift-value: 114.286, RR: 33.3333)
b <- a c (sup: 40, conf: 100, lift-value: 142.857, RR: 100)
b <- a (sup: 50, conf: 83.3333, lift-value: 119.048, RR: 44.4444)
c <- b (sup: 60, conf: 85.7143, lift-value: 122.449, RR: 52.381)
b <- c (sup: 60, conf: 85.7143, lift-value: 122.449, RR: 52.381)



In [244]:
os.system('apriori.exe -trs10m2n5 -el -v " (sup: %S, conf: %C, lift-value: %E)" datasets/exemple-1.txt res.txt')
print(open("res.txt", "r").read())

d <- e a (sup: 10, conf: 100, lift-value: 142.857)
d <- e c (sup: 10, conf: 100, lift-value: 142.857)
d <- e (sup: 30, conf: 100, lift-value: 142.857)
b <- a c d (sup: 20, conf: 100, lift-value: 142.857)
c <- a b (sup: 40, conf: 80, lift-value: 114.286)
b <- a c (sup: 40, conf: 100, lift-value: 142.857)
b <- a (sup: 50, conf: 83.3333, lift-value: 119.048)
c <- b (sup: 60, conf: 85.7143, lift-value: 122.449)
b <- c (sup: 60, conf: 85.7143, lift-value: 122.449)



In [243]:
os.system('apriori.exe -trs10m2n5 -ez -v " (sup: %S, conf: %C, RR: %E)" datasets/exemple-1.txt res.txt')
print(open("res.txt", "r").read())

d <- e a (sup: 10, conf: 100, RR: 100)
d <- e c (sup: 10, conf: 100, RR: 100)
d <- e (sup: 30, conf: 100, RR: 100)
b <- a c d (sup: 20, conf: 100, RR: 100)
c <- a b (sup: 40, conf: 80, RR: 33.3333)
b <- a c (sup: 40, conf: 100, RR: 100)
b <- a (sup: 50, conf: 83.3333, RR: 44.4444)
c <- b (sup: 60, conf: 85.7143, RR: 52.381)
b <- c (sup: 60, conf: 85.7143, RR: 52.381)



- Base mushrooms

In [242]:
os.system('apriori.exe -trs90m2n5c90 -el -ez -v " (sup: %S, conf: %C, lift-value: %L, RR: %E)" datasets/mushrooms.txt res.txt')
print(open("res.txt", "r").read())

PARTIAL <- ONE FREE WHITE (sup: 90.019, conf: 100, lift-value: 100, RR: nan)
WHITE <- ONE FREE PARTIAL (sup: 90.019, conf: 100, lift-value: 102.335, RR: 100)
FREE <- ONE WHITE PARTIAL (sup: 90.019, conf: 100, lift-value: 102.634, RR: 100)
WHITE <- ONE FREE (sup: 90.019, conf: 100, lift-value: 102.335, RR: 100)
FREE <- ONE WHITE (sup: 90.019, conf: 100, lift-value: 102.634, RR: 100)
PARTIAL <- ONE FREE (sup: 90.019, conf: 100, lift-value: 100, RR: nan)
PARTIAL <- ONE WHITE (sup: 90.019, conf: 100, lift-value: 100, RR: nan)
PARTIAL <- ONE (sup: 92.3004, conf: 100, lift-value: 100, RR: nan)
PARTIAL <- FREE WHITE (sup: 97.4335, conf: 100, lift-value: 100, RR: nan)
WHITE <- FREE PARTIAL (sup: 97.4335, conf: 100, lift-value: 102.335, RR: 100)
FREE <- WHITE PARTIAL (sup: 97.4335, conf: 99.7082, lift-value: 102.335, RR: 88.6295)
WHITE <- FREE (sup: 97.4335, conf: 100, lift-value: 102.335, RR: 100)
FREE <- WHITE (sup: 97.4335, conf: 99.7082, lift-value: 102.335, RR: 88.6295)
PARTIAL <- FREE (su

In [241]:
os.system('apriori.exe -trs90m2n5c90 -el -v " (sup: %S, conf: %C, lift-value: %L)" datasets/mushrooms.txt res.txt')
print(open("res.txt", "r").read())

PARTIAL <- ONE FREE WHITE (sup: 90.019, conf: 100, lift-value: 100)
WHITE <- ONE FREE PARTIAL (sup: 90.019, conf: 100, lift-value: 102.335)
FREE <- ONE WHITE PARTIAL (sup: 90.019, conf: 100, lift-value: 102.634)
ONE <- FREE WHITE PARTIAL (sup: 90.019, conf: 92.3902, lift-value: 100.097)
WHITE <- ONE FREE (sup: 90.019, conf: 100, lift-value: 102.335)
FREE <- ONE WHITE (sup: 90.019, conf: 100, lift-value: 102.634)
ONE <- FREE WHITE (sup: 90.019, conf: 92.3902, lift-value: 100.097)
PARTIAL <- ONE FREE (sup: 90.019, conf: 100, lift-value: 100)
FREE <- ONE PARTIAL (sup: 90.019, conf: 97.5283, lift-value: 100.097)
ONE <- FREE PARTIAL (sup: 90.019, conf: 92.3902, lift-value: 100.097)
FREE <- ONE (sup: 90.019, conf: 97.5283, lift-value: 100.097)
ONE <- FREE (sup: 90.019, conf: 92.3902, lift-value: 100.097)
PARTIAL <- ONE WHITE (sup: 90.019, conf: 100, lift-value: 100)
WHITE <- ONE PARTIAL (sup: 90.019, conf: 97.5283, lift-value: 99.8052)
ONE <- WHITE PARTIAL (sup: 90.019, conf: 92.1206, lift-v

In [247]:
os.system('apriori.exe -trs90m2n5c90 -ez -v " (sup: %S, conf: %C, RR: %E)" datasets/mushrooms.txt res.txt')
print(open("res.txt", "r").read())

PARTIAL <- ONE FREE WHITE (sup: 90.019, conf: 100, RR: nan)
WHITE <- ONE FREE PARTIAL (sup: 90.019, conf: 100, RR: 100)
FREE <- ONE WHITE PARTIAL (sup: 90.019, conf: 100, RR: 100)
WHITE <- ONE FREE (sup: 90.019, conf: 100, RR: 100)
FREE <- ONE WHITE (sup: 90.019, conf: 100, RR: 100)
PARTIAL <- ONE FREE (sup: 90.019, conf: 100, RR: nan)
PARTIAL <- ONE WHITE (sup: 90.019, conf: 100, RR: nan)
PARTIAL <- ONE (sup: 92.3004, conf: 100, RR: nan)
PARTIAL <- FREE WHITE (sup: 97.4335, conf: 100, RR: nan)
WHITE <- FREE PARTIAL (sup: 97.4335, conf: 100, RR: 100)
FREE <- WHITE PARTIAL (sup: 97.4335, conf: 99.7082, RR: 88.6295)
WHITE <- FREE (sup: 97.4335, conf: 100, RR: 100)
FREE <- WHITE (sup: 97.4335, conf: 99.7082, RR: 88.6295)
PARTIAL <- FREE (sup: 97.4335, conf: 100, RR: nan)
PARTIAL <- WHITE (sup: 97.7186, conf: 100, RR: nan)



- Base titanic

In [248]:
os.system('apriori.exe -trs40m2n5c50 -v " (sup: %S, conf: %C, lift-value: %L, RR: %E)" datasets/titanic-red.csv res.txt')
print(open("res.txt", "r").read())

female <- alive (sup: 32.3887, conf: 69.7674, lift-value: 179.506, RR: 0)
Southampton <- alive (sup: 29.8246, conf: 64.2442, lift-value: 90.332, RR: 0)
Southampton <- dead male (sup: 36.8421, conf: 78.4483, lift-value: 110.304, RR: 0)
male <- dead Southampton (sup: 36.8421, conf: 89.2157, lift-value: 146.258, RR: 0)
dead <- male Southampton (sup: 36.8421, conf: 79.5918, lift-value: 148.933, RR: 0)
male <- dead (sup: 46.9636, conf: 87.8788, lift-value: 144.067, RR: 0)
dead <- male (sup: 46.9636, conf: 76.9912, lift-value: 144.067, RR: 0)
Southampton <- dead (sup: 41.2955, conf: 77.2727, lift-value: 108.651, RR: 0)
dead <- Southampton (sup: 41.2955, conf: 58.0645, lift-value: 108.651, RR: 0)
Southampton <- male (sup: 46.2888, conf: 75.885, lift-value: 106.7, RR: 0)
male <- Southampton (sup: 46.2888, conf: 65.0854, lift-value: 106.7, RR: 0)



In [249]:
os.system('apriori.exe -trs40m2n5c50 %l -el -v " (sup: %S, conf: %C, lift-value: %L)" datasets/titanic-red.csv res.txt')
print(open("res.txt", "r").read())

female <- alive (sup: 32.3887, conf: 69.7674, lift-value: 179.506, RR: 0)
Southampton <- alive (sup: 29.8246, conf: 64.2442, lift-value: 90.332, RR: 0)
Southampton <- dead male (sup: 36.8421, conf: 78.4483, lift-value: 110.304, RR: 0)
male <- dead Southampton (sup: 36.8421, conf: 89.2157, lift-value: 146.258, RR: 0)
dead <- male Southampton (sup: 36.8421, conf: 79.5918, lift-value: 148.933, RR: 0)
male <- dead (sup: 46.9636, conf: 87.8788, lift-value: 144.067, RR: 0)
dead <- male (sup: 46.9636, conf: 76.9912, lift-value: 144.067, RR: 0)
Southampton <- dead (sup: 41.2955, conf: 77.2727, lift-value: 108.651, RR: 0)
dead <- Southampton (sup: 41.2955, conf: 58.0645, lift-value: 108.651, RR: 0)
Southampton <- male (sup: 46.2888, conf: 75.885, lift-value: 106.7, RR: 0)
male <- Southampton (sup: 46.2888, conf: 65.0854, lift-value: 106.7, RR: 0)



In [250]:
os.system('apriori.exe -trs40m2n5c50 -ez -v " (sup: %S, conf: %C, RR: %E)"datasets/titanic-red.csv res.txt')
print(open("res.txt", "r").read())

female <- alive (sup: 32.3887, conf: 69.7674, lift-value: 179.506, RR: 0)
Southampton <- alive (sup: 29.8246, conf: 64.2442, lift-value: 90.332, RR: 0)
Southampton <- dead male (sup: 36.8421, conf: 78.4483, lift-value: 110.304, RR: 0)
male <- dead Southampton (sup: 36.8421, conf: 89.2157, lift-value: 146.258, RR: 0)
dead <- male Southampton (sup: 36.8421, conf: 79.5918, lift-value: 148.933, RR: 0)
male <- dead (sup: 46.9636, conf: 87.8788, lift-value: 144.067, RR: 0)
dead <- male (sup: 46.9636, conf: 76.9912, lift-value: 144.067, RR: 0)
Southampton <- dead (sup: 41.2955, conf: 77.2727, lift-value: 108.651, RR: 0)
dead <- Southampton (sup: 41.2955, conf: 58.0645, lift-value: 108.651, RR: 0)
Southampton <- male (sup: 46.2888, conf: 75.885, lift-value: 106.7, RR: 0)
male <- Southampton (sup: 46.2888, conf: 65.0854, lift-value: 106.7, RR: 0)



<font size="+1">**[Q]**</font>  Ajouter les deux mesures d'intérêt suivantes et afficher leurs valeurs
  pour chaque règle trouvée. Ajouter un argument au programme afin de pouvoir sélectionner une de ces mesures pour éliminer les règles inintéressantes.
  
Par exemple:

\begin{align}
  \mbox{Interest}(X \longrightarrow Y) & =  \frac{P(X,Y)}{P(X)}P(Y) \nonumber\\
 \mbox{IS}(X \longrightarrow Y) & =  \frac{P(X,Y)}{\sqrt{P(X)P(Y)}} \nonumber
\end{align}

In [306]:
import numpy as np

def interest(BASE, X, Y):
    return support(BASE, X.union(Y)) / support(BASE, X) * support(BASE, Y)

def IS(BASE, X, Y):
    return support(BASE, X.union(Y)) / np.sqrt(support(BASE, X) * support(BASE, Y))

def showRules2(filename, minsupp, minconf=80, measure="x", sep=" "):
    BASE = chargeBase(filename, sep)
    os.system('apriori.exe -trs{}m2n5c{} -e{} -v " (%X %C" {} res.txt'.format(minsupp, minconf, measure, filename))
    rules = open("res.txt", "r").read().split("\n")[:-1]
    for rule in rules:
        X = {rule.split("<-")[0].strip()}
        Y = rule.split("<-")[1].split("(")[0]
        Y = set(Y.split(" ")[1:-1])
        supp, conf = rule.split("(")[1].split(" ")
        print( "{} <- {} (supp: {}, conf: {}, interest: {}, IS: {})".format(X, Y, supp, conf, 
                                                                          interest(BASE, X, Y), 
                                                                          IS(BASE, X, Y)) )

- Base exemple-1

In [307]:
showRules2("datasets/exemple-1.txt", 10)

{'d'} <- {'a', 'e'} (supp: 10, conf: 100, interest: 0.014285714285714289, IS: 0.3779644730092273)
{'d'} <- {'c', 'e'} (supp: 10, conf: 100, interest: 0.014285714285714289, IS: 0.3779644730092273)
{'d'} <- {'e'} (supp: 30, conf: 100, interest: 0.1285714285714286, IS: 0.6546536707079771)
{'b'} <- {'d', 'c', 'a'} (supp: 20, conf: 100, interest: 0.057142857142857155, IS: 0.5345224838248488)
{'c'} <- {'b', 'a'} (supp: 50, conf: 80, interest: 0.28571428571428575, IS: 0.6761234037828133)
{'b'} <- {'c', 'a'} (supp: 40, conf: 100, interest: 0.22857142857142862, IS: 0.7559289460184546)
{'b'} <- {'a'} (supp: 60, conf: 83.3333, interest: 0.42857142857142855, IS: 0.7715167498104596)
{'c'} <- {'b'} (supp: 70, conf: 85.7143, interest: 0.6, IS: 0.8571428571428572)
{'b'} <- {'c'} (supp: 70, conf: 85.7143, interest: 0.6, IS: 0.8571428571428572)


In [310]:
showRules2("datasets/titanic-red.csv", 30, sep=",")

{'Southampton'} <- {'2nd'} (supp: 34.8178, conf: 87.9845, interest: 0.1499742642257373, IS: 0.6156171310119235)
{'alive'} <- {'female'} (supp: 38.8664, conf: 83.3333, interest: 0.27116090763581585, IS: 0.7624928516630233)
{'male'} <- {'dead', 'Southampton'} (supp: 41.2955, conf: 89.2157, interest: 0.24941779226828134, IS: 0.7340619556352773)
{'male'} <- {'dead'} (supp: 53.4413, conf: 87.8788, interest: 0.41145068252660244, IS: 0.8225502403062295)


- Base mushrooms

- Base titanic

### Utilisation de fp-growth

<font size="+1">**[Q]**</font>  Utiliser *fpgrowth32* pour générer des règles d'association sur les différentes
  bases de données. Comparer avec les résultats obtenus dans la section précédente. En  particulier, comparer les temps d'éxecution.

In [332]:
from time import time

print("Base titanic minsupp=10")
start = time()
os.system("./apriori.exe -trs10m2n5 datasets/titanic-red.csv")
end = time()
print("Temps apriori :", end-start)

start = time()
os.system("./fpgrowth.exe -trs10m2n5 datasets/titanic-red.csv")
end = time()
print("Temps fpgrowth :", end-start)

Base titanicc minsupp=10
Temps apriori : 0.04353070259094238
Temps fpgrowth : 0.03202247619628906


In [351]:
from time import time

print("Base mushrooms minsupp=10")
start = time()
os.system("./apriori.exe -trs1m2n5 datasets/mushrooms.txt")
end = time()
print("Temps apriori :", end-start)

start = time()
os.system("./fpgrowth.exe -trs1m2n5 datasets/mushrooms.txt")
end = time()
print("Temps fpgrowth :", end-start)

Base mushrooms minsupp=10
Temps apriori : 0.13526129722595215
Temps fpgrowth : 0.02979755401611328


<font size="+1">**[Q]**</font> En essayant différentes valeurs de seuil, tester la génération de règles d'association intéressantes sur les bases de données fournies.

## Annexes: utilisation des programmes *apriori32* et *fpgrowth32*

La documentation complète de ces programmes est disponible aux URL suivantes:
- pour *apriori*: http://www.borgelt.net/doc/apriori/apriori.html
- pour *fp-growth*: http://www.borgelt.net/doc/fpgrowth/fpgrowth.html


Leur format d'utilisation général est:

        ./<programme> [options] infile [outfile]

(où <<code>programme</code>> est soit *apriori32*, soit *fpgrowth32*).

Les options de base de ces 2 programmes sont:
- sans argument: génération des itemsets fréquents (argument {<code>-ts</code> activé par défaut)
- *tr*: pour obtenir les règles d'associations
- *s*: pour fournir une valeur minimale de support. Le support est ici donné en valeur absolue ($n_{AB}$) et non pas en valeur relative ($\frac{n_{AB}}{n}$).
- *m*: pour fournir un nombre minimum d'items dans un itemset
- *n*: pour fournir un nombre maximum d'items dans un itemset


Par exemple (commandes lancées dans le répertoire père du répertoire *datasets/*):

        ./apriori32 -trs50m2n5 datasets/exemple-1.txt fichier-resultat.out  
        ./fpgrowth32 -trs50m2n5 datasets/mushrooms.txt fichier-resultat.out  
